In [113]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoProcessor, Trainer, TrainingArguments, TrainerCallback
from datasets import load_dataset
import lion_pytorch
from IPython.display import clear_output
import pandas as pd

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Выбор модели

В качестве основы я выбрал новую модель Florence2-base от Microsoft, и это не случайно. Есть несколько причин для этого выбора:

1. Малое количество  параметров: Florence2 имеет всего 270 миллионов параметров, что облегчает ее дообучение. Благодаря этому процесс вывода и квантования в этой модели будет проще, чем в более массивных вариантах.
   
2. Обильные данные: Florence2 обучалась на данных, включающих 126 миллионов изображений в различных задачах, таких как классификация, сегментация и обнаружение объектов. Это гарантирует более высокую эффективность модели в задаче CQA.

Сама структура модели немного отличается от стандартной VLM: в то время как большинство VLM состоят из энкодера и декодера, Florence2 включает в себя три ключевые компоненты - Визуальный Энкодер, Энкодер Вопросов и Декодер. 

Для получения эмбеддингов изображений используется DaViT. Эмбеддинги из визуального энкодера затем проходят через энкодер вопросов, где комбинируются с токенами. Затем эмбеддинги вопросов поступают в декодер, который завершает моделирование и генерацию текста


In [4]:
model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-base-ft", attn_implementation="flash_attention_2", trust_remote_code=True)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base-ft", trust_remote_code=True)
path = '/home/oleg/models/checkpoint-1000/'
model = AutoModelForCausalLM.from_pretrained(path, config=model.config, attn_implementation="flash_attention_2", use_safetensors=True, trust_remote_code=True).cuda()
count_parameters(model)

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in Florence2ForConditionalGeneration is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`


270803968

# Выбор датасета

Для процесса настройки я решил использовать датасет ChartQA от команды HuggingFaceM, и в этом есть несколько причин:

1. Объемный датасет: Содержащий 27 тысяч элементов, этот датасет обеспечивает достаточно данных для успешного дообучения модели на аналогичную задачу.
   
2. Предыдущий опыт модели: Учитывая то, что модель была обучена на задачах детекции и сегментации, это позволит ей более точно определять содержимое изображений, что, в свою очередь, повысит точность ответов модели.


In [5]:
def fn(data):
    image = data['image'].convert("RGB")
    query = data['query']
    label = data['label']

    out = processor(text=query, images=image, return_tensors="pt", padding=False)
    label = processor.tokenizer(text=label, return_tensors='pt', padding=False)['input_ids']

    data['input_ids'] = out['input_ids'].squeeze(0)
    data['pixel_values'] = out['pixel_values'].squeeze(0)
    data['labels'] = label.squeeze(0)
    return data

In [6]:
ds = load_dataset("HuggingFaceM4/ChartQA", streaming=True)['train']
ds = ds.map(fn, batched=False, remove_columns=['image', 'human_or_machine', 'query', 'label'])

# Обучение

Для сохранения ранее полученных знаний модели я принял решение заморозить визуальный энкодер. 

1. В качестве оптимизатора был выбран Lion с параметрами: learning rate = 2e-5,  betas = (0.9, 0.99) и weight decay = 2.5e-2.

2. Был выбран размер пакета  равный 1024 и выполнено 100 шагов, что приблизительно эквивалентно 10 эпохам обучения.


In [7]:
for param in model.vision_tower.parameters():
  param.is_trainable = False

In [8]:
class BatchLossCallback(TrainerCallback):
    def __init__(self):
        self.steps = 0
        self.loss = 0
        self.epoch = 0.0
    
    def on_step_end(self, args, state, control, **kwargs):
        clear_output(wait=True)
        if state.log_history:
            print(f"Batch {state.global_step}: Loss = {state.log_history[-1]}")
            self.steps +=1
            self.loss += state.log_history[-1]['loss']

In [ ]:
optimizer = lion_pytorch.Lion(model.parameters(),lr=2e-5, betas=(0.9, 0.99), weight_decay=2.5e-2, use_triton=True)


training_args = TrainingArguments(
    output_dir="/home/oleg/models/",  
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1024,  
    max_steps=100,
    gradient_checkpointing=False,
    fp16=True,
    warmup_steps=0,
    evaluation_strategy="no",
    save_steps= 1,
    do_eval = False,
    logging_steps=1,    
    max_grad_norm =0.75,
    optim="adamw_torch_fused",
    learning_rate=2e-4,
    weight_decay=1e-2,
    adam_beta1=0.91,
    adam_beta2=0.98,
    adam_epsilon=1e-8,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    remove_unused_columns=False

)

trainer = Trainer(
    args=training_args,
    model=model,
    train_dataset=ds,
    optimizers=(optimizer,None),
    callbacks=[BatchLossCallback()]
)

trainer.train()

# Сравнение моделей до и после обучения

Для оценки качества я решил использовать метрику WER (Word Error Rate). Хотя обычно WER применяется в задачах распознавания речи, в данном датасете ответы состоят либо из одного слова, либо из одного числа. Поэтому WER будет работать аналогично точности и может показать себя более эффективно, чем другие метрики

In [10]:

path = '/home/oleg/models/checkpoint-100/'
orig_model =  AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-base-ft", attn_implementation="flash_attention_2", trust_remote_code=True).eval().cuda()
tuned_model = AutoModelForCausalLM.from_pretrained(path, config=model.config, use_safetensors=True, trust_remote_code=True).eval().cuda()


In [11]:
ds = load_dataset("HuggingFaceM4/ChartQA", streaming=True)['test']
ds = ds.map(fn, batched=False, remove_columns=['image', 'human_or_machine', 'query', 'label'])
dataloader = DataLoader(ds, batch_size=1)

In [28]:
pred_tuned = []
pred_orig = []
references = []
for idx, batch in enumerate(dataloader):

    input_ids = batch['input_ids'].cuda()
    pixel_values = batch['pixel_values'].cuda()
    labels = batch['labels'].cuda()
    l = labels.shape[1]
    with torch.no_grad():
        out_tuned = tuned_model.generate(input_ids=input_ids, pixel_values=pixel_values, max_new_tokens=l)
        out_orig = orig_model.generate(input_ids=input_ids, pixel_values=pixel_values, max_new_tokens=l)
    s_tuned = processor.tokenizer.decode(out_tuned.cpu()[0, :], skip_special_tokens=True)
    s_orig = processor.tokenizer.decode(out_orig.cpu()[0, :], skip_special_tokens=True)
    s = processor.tokenizer.decode(labels.cpu()[0, :], skip_special_tokens=True)
    pred_tuned.append(s_tuned)
    pred_orig.append(s_orig)
    references.append(s)
    clear_output(True)
    print(f'{ idx} ')

    

2499 


In [111]:
from evaluate import load
wer = load("wer")

orig_wer_score = wer.compute(predictions=pred_orig, references=references)
tuned_wer_score = wer.compute(predictions=pred_tuned, references=references)
print(orig_wer_score)
print(tuned_wer_score )

0.9884816753926702
0.09598603839441536


In [114]:
df = pd.DataFrame(list(zip(pred_orig, pred_tuned, references)), columns=['orig', 'tuned', 'references'])
df.to_csv('predictions.csv', index=False)

# Результаты

В процессе проведения исследования удалось значительно улучшить показатели модели с 0.98 до 0.1. Полученные результаты свидетельствуют о том, что во многих задачах обработки визуальных вопросов и ответов (VQA) использование огромных языковых моделей не является обязательным. Достаточно использовать меньшие модели с хорошо отфильтрованными данными.